<a href="https://colab.research.google.com/github/Aleena9k/Travel_Agent/blob/main/Crewai_travel_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-1: Install the packages

In [1]:
!pip install crewai
!pip install crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.1/327.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.7 MB/s 

#Step-2: Import packages

In [2]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


#Step-3: Access the keys

In [3]:
from google.colab import userdata
gemini_key = userdata.get('gemini_key')
serper_key = userdata.get('serper_key')
os.environ['GOOGLE_API_KEY'] = gemini_key
os.environ['SERPER_API_KEY'] = serper_key

#Step-4: LLM and Tool

In [4]:
#Inititalize a search tool(to fetch real-time travel info)
search_tool = SerperDevTool()
#Deffine the AI Model
llm = LLM(model= "gemini/gemini-2.5-flash", #model name
          verbose = True, #I want to see the statements
          temperature=0.5, #thinking power
          api_key = os.environ["GOOGLE_API_KEY"])
llm

#Step-5: Initialize the agents

In [5]:
#Travel Reasearcher Agent(find historical sites + weather)
researcher = Agent(
    role = "Travel Researcher",
    goal = "Find historical sites, public transport hotels, and real time weather for {destination}.",
    verbose = True, #I want to see when the code runs
    memory =True, #store
    backstory = "You are an expert travel researcher, providing up-to-date information about history focused trips",
    llm = llm,
    tools =[search_tool], #Uses live search tool
    allow_delegation = True #Transfer the task ahead
)

#Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role = "Budget Planner",
    goal = "Find budget flights, hotels, and activities within {budgets} for {destination}",
    verbose = True,
    memory =True,
    backstory = "You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm = llm,
    tools =[search_tool],
    allow_delegation = True
)

#Itinerary Planner Agent (Creates a balanced 3-day plan)

itinerary_planner = Agent(
    role = "Itinerary Researcher",
    goal = "Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budgets}.",
    verbose = True,
    memory =True,
    backstory = "You are an expert in trip planning, ensuring travellers get the best experience within their budget",
    llm = llm,
    tools =[search_tool],
    allow_delegation = False
)

#Step-6: Agent tasks

In [6]:
#Travel Research Task
researcher_task = Task(
    description = "Find the best historical sites, weather forecast and public transport hotels for {destination}",
    expected_output= "A list of top historical sites, a real-time weather update, and 3 hotel options near public transport",
    tools = [search_tool],
    agent = researcher
)

#Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budgets}",
    expected_output="A full cost breakdown (flights, hotel, food, attraction) ensuring a $1500 budget is maintained.",
    tools = [search_tool],
    agent = budget_planner
)

#Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather updates.",
    expected_output="A detailed 3-day plan, considering weather and buget constraints, with transport recommendations.",
    tools = [search_tool],
    agent = itinerary_planner
)

#Step-7: Kick Off the work

In [7]:
#Crew Setup: All agents working together
crew = Crew(
    agents = [researcher, budget_planner, itinerary_planner],
    tasks = [researcher_task, budget_task, itinerary_task],
    process = Process.sequential # Runs tasks in sequence
)
result = crew.kickoff(inputs={'destination': 'Hyderabad',
                              'budgets': '25000'})
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Task: Find the best historical sites, weather forecast and public transport hotels for Hyderabad               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"top historical sites in Hyderabad\"}"                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'top historical sites in Hyderabad', 'type': 'search', 'num': 10, 'engine':         │
│  'google'}, 'organic': [{'title': 'THE 10 BEST Hyderabad Sights & Historical Landmarks to Visit (2025)',        │
│  'link':                                                                                                        │
│  'https://www.tripadvisor.com/Attractions-g297586-Activities-c47-Hyderabad_Hyderabad_District_Telangana.html',  │
│  'snippet': 'Charminar.', 'position': 1}, {'title': '20 Amazing Historical Places in Hyderabad to Know its      │
│  Past!', 'link': 'https://wanderon.in/blogs/historical-places-in-hyderabad', 'snippet': '1. Charminar · 2.      │
│  Golconda Fort · 3. Qutb Shahi Tombs · 4. Chowmahalla Palace · 5. Mecca Masjid · 6. Falaknuma Palace · 7.       │
│  Paigah Tombs · 8. Salar Jung ...', 'position': 2}, {'title': 'Top 5 places to visit in Hyderabad for history   │
│  lovers - MySoulTravels', 'link':                                                                               │
│  'https://www.mysoultravels.com/india/telangana/places-to-visit-in-hyderabad/', 'snippet': 'Historical          │
│  landmarks like the Mecca Masjid, Char Kaman and the old hospital buildings add to the heritage scene of        │
│  Charminar. The Mecca Masjid is one of the ...', 'position': 3}, {'title': 'Best historical places in and       │
│  around Hyderabad - Reddit', 'link':                                                                            │
│  'https://www.reddit.com/r/hyderabad/comments/1iyxdnq/best_historical_places_in_and_around_hyderabad/',         │
│  'snippet': 'Apart from Golconda fort, Qutub Shahi tombs, Charminar, Warangal fort, Ramappa temple. Are there   │
│  any other interesting sites?', 'position': 4}, {'title': 'Famous Historical Places in Hyderabad | Thorn Tree   │
│  Travel Forum', 'link': 'https://www.thorntreeforum.com/threads/famous-historical-places-in-hyderabad.848/',    │
│  'snippet': 'Qutb Shahi Tombs: Located near Golconda Fort, these tombs reflect Persian, Pashtun, and Hindu      │
│  influences. The tranquil gardens and meticulously ...', 'position': 5}, {'title': 'Hyderabad Heritage -        │
│  Incredible India', 'link': 'https://www.incredibleindia.gov.in/en/trips/trip-listing/hyderabad-heritage',      │
│  'snippet': 'Start your Hyderabad adventure with a day Chowmahalla Palace, a splendid architectural             │
│  masterpiece that was o...                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Thought: Thought: I have a good list of historical sites. Now I need to find the real-time weather update for  │
│  Hyderabad and then look for hotels near public transport.                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Salar Jung Museum entrance fee for Indian citizens', 'type': 'search', 'num': 10,  │
│  'engine': 'google'}, 'organic': [{'title': 'Salar Jung Museum, Hyderabad, Telangana, India', 'link':           │
│  'https://salarjungmuseum.in/Hours-and-admission.html', 'snippet': 'Entry Fee · 50/- per Adult (above 18        │
│  Years) for Indians · 20/- per Children (above 5 years) for Indians · 500/- per person for any Foreign Visitor  │
│  · 50/- per Still ...', 'position': 1}, {'title': 'Salar Jung Museum, Hyderabad - Ticket Price, Timings,        │
│  Photos', 'link': 'https://www.makemytrip.com/tripideas/attractions/salar-jung-museum', 'snippet': 'The Salar   │
│  Jung Museum ticket prices are as follows. Indian Adults: INR 20. Children (below 18 years): INR 10.            │
│  Foreigners: INR 500 · 10:00 AM to 5:00 PM. It ...', 'position': 2}, {'title': 'Salar Jung Museum Hyderabad,    │
│  timings, entry ticket cost, price, fee', 'link':                                                               │
│  'https://www.hyderabadtourism.travel/salar-jung-museum-hyderabad', 'snippet': 'Entry Fee- The entry fee for    │
│  Salarjung Museum is Rs 50 for Indian adults and Rs. 20 for Indian children above 5 years of age. For students  │
│  and children ...', 'position': 3}, {'title': 'Salarjung Museum Hyderabad | Entry Fee, Timings,Location ...',   │
│  'link':                                                                                                        │
│  'https://weekendyaari.in/2024/11/09/salarjung-museum-hyderabad-entry-fee-timingslocation-contact-number/',     │
│  'snippet': 'Salarjung Museum Entry Fee ; Indian Adults, ₹20 ; Foreign Nationals, ₹500 ; Children (under 12     │
│  years), ₹10 ; School Groups (with permission), ₹5.', 'position': 4}, {'title': "Citizen's Charter - Hyderabad  │
│  - Salar Jung Museum", 'link': 'https://salarjungmuseum.in/citizens-charter.html', 'snippet': '10.00 A.M. to    │
│  5.00 P.M.. The Museum is closed on Fridays and Gazetted holidays. ADMISSION TICKETS: 50/- per Adult (above 18  │
│  Years) for Indians', 'position': 5}, {'title': 'Salar Jung Museum (2025) - All You Need to Know ... -          │
│  Tripadvisor', 'link':                                                                                          │
│  'https://www.tripadvisor.com/Attraction_Review-g297586-d320681-Reviews-Salar_Jung_Museum-Hyderabad_Hy...       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Planner                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Budget Trip Plan for Hyderabad (3 Days)**                                                                    │
│                                                                                                                 │
│  **Total Estimated Cost: INR 10,595**                                                                           │
│                                                                                                                 │
│  This plan ensures the total cost stays well under the 25,000 INR budget, providing a comfortable yet           │
│  economical travel experience.                                                                                  │
│                                                                                                                 │
│  **Cost Breakdown:**                                                                                            │
│                                                                                                                 │
│  1.  **Flights (Round Trip from Delhi to Hyderabad):**                                                          │
│      *   **Estimated Cost:** INR 7,000                                                                          │
│      *   *Details:* Based on current budget airline prices, with options from carriers like IndiGo, Air India,  │
│  and Akasa Air. Prices can vary, but searching for fares around ₹7,000 - ₹8,500 is feasible for a round trip.   │
│                                                                                                                 │
│  2.  **Accommodation (3 Nights):**                                                                              │
│      *   **Hotel Option:** SPOT ON Hotel Rk Residency Near MGBS Bus station                                     │
│      *   **Estimated Daily Cost:** INR 600                                                                      │
│      *   **Total Hotel Cost:** INR 600/night * 3 nights = **INR 1,800**                                         │
│      *   *Details:* This hotel is chosen for its budget-friendly pricing and convenient access to public        │
│  transport (MGBS Bus station).                                                                                  │
│                                                                                                                 │
│  3.  **Food (3 Days):**                                                                                         │
│      *   **Estimated Daily Cost:** INR 400                                                                      │
│      *   **Total Food Cost:** INR 400/day * 3 days = **INR 1,200**                                              │
│      *   *Details:* This budget allows for eating at local eateries, street food stalls, and small              │
│  restaurants, which offer authentic and affordable Hyderabadi cuisine.                                          │
│                                                                                                                 │
│  4.  **Local Transport (3 Days):**                                                                              │
│      *   **Estimated Daily Cost:** INR 150                                                                      │
│      *   **Total Transport Cost:** INR 150/day * 3 days

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Itinerary Researcher                                                                                    │
│                                                                                                                 │
│  Task: Plan a 3-day itinerary for Hyderabad, focusing on historical sites, budget constraints, and real-time    │
│  weather updates.                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Itinerary Researcher                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **3-Day Budget-Friendly Historical Itinerary for Hyderabad**                                                   │
│                                                                                                                 │
│  **Budget Overview:**                                                                                           │
│  *   **Total Estimated Cost:** INR 10,595 (well within the INR 25,000 limit)                                    │
│  *   **Accommodation:** SPOT ON Hotel Rk Residency Near MGBS Bus station (chosen for budget and public          │
│  transport access).                                                                                             │
│                                                                                                                 │
│  **Weather Consideration:**                                                                                     │
│  As of the current update, Hyderabad is 26°C (82°F), mostly cloudy with light rain, and feels like 31°C (87°F)  │
│  due to 79% humidity.                                                                                           │
│  *   **Recommendation:** Carry a light umbrella or raincoat. Wear comfortable, breathable clothing              │
│  (cotton/linen) and sturdy walking shoes. Stay hydrated throughout the day. The light rain can offer some       │
│  respite from the heat, but humidity will be noticeable.                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Detailed Itinerary:**                                                                                        │
│                                                                                                                 │
│  **Day 1: Old City Charm & Heritage**                                                                           │
│                                                                                                                 │
│  *   **Morning (9:00 AM - 1:00 PM): Charminar, Mecca Masjid & Local Market**                                    │
│      *   **Activity:** Start your day early at **Charminar**, the iconic symbol of Hyderabad. Spend time        │
│  admiring its architecture and the bustling surroundings. Just adjacent is **Mecca Masjid**, one of India's     │
│  largest mosques; explore its serene interiors (dress modestly).                                                │
│      *   **Transport:** From SPOT ON Hotel Rk Residency (near MGBS Bus Station), take a local TSRTC bus         │
│  towards Charminar. Shared auto-rickshaws are also readily available.                                           │
│      *   **Entrance Fee:** Charminar: INR 5 (for Indian citizens). Mecca Masjid: Free.                          │
│      *   **Food:** Explore the vibrant street food scene around Charminar for a traditional Hyderabadi          │
│  breakfast (e.g., Irani Chai with Osmania Biscuits, or local dosas/idlis). This will be budget-friendly.        │
│                                                        

**3-Day Budget-Friendly Historical Itinerary for Hyderabad**

**Budget Overview:**
*   **Total Estimated Cost:** INR 10,595 (well within the INR 25,000 limit)
*   **Accommodation:** SPOT ON Hotel Rk Residency Near MGBS Bus station (chosen for budget and public transport access).

**Weather Consideration:**
As of the current update, Hyderabad is 26°C (82°F), mostly cloudy with light rain, and feels like 31°C (87°F) due to 79% humidity.
*   **Recommendation:** Carry a light umbrella or raincoat. Wear comfortable, breathable clothing (cotton/linen) and sturdy walking shoes. Stay hydrated throughout the day. The light rain can offer some respite from the heat, but humidity will be noticeable.

---

**Detailed Itinerary:**

**Day 1: Old City Charm & Heritage**

*   **Morning (9:00 AM - 1:00 PM): Charminar, Mecca Masjid & Local Market**
    *   **Activity:** Start your day early at **Charminar**, the iconic symbol of Hyderabad. Spend time admiring its architecture and the bustling surroundin